In [7]:
import torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import torchvision as tv
import torchvision.transforms as transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt



###UP IS ORIGIN

import math
import torch.optim as optim


### SEC 3
from torchvision.utils import save_image






# custom dataloader for .npy file
class numpyDataset(Dataset):
    def __init__(self, data, transform=None):
        self.data = torch.from_numpy(data).float()
        self.transform = transform
        
    def __getitem__(self, index):
        x = self.data[index]
        
        if self.transform:
            x = self.transform(x)
        
        return x
    
    def __len__(self):
        return len(self.data)


In [8]:
### Setting up Device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
# Below codes activates when want to use cpu
#device = torch.device('cpu')
print('Using device:', device)
print()

#Additional Info when using cuda
if device.type == 'cuda':
    print(torch.cuda.get_device_name(0))
    print('Memory Usage:')
    print('Allocated:', round(torch.cuda.memory_allocated(0)/1024**3,1), 'GB')
    print('Cached:   ', round(torch.cuda.memory_cached(0)/1024**3,1), 'GB')

Using device: cuda

GeForce GTX 1050
Memory Usage:
Allocated: 0.4 GB
Cached:    1.1 GB


In [9]:
transform       = transforms.Compose([
                                transforms.ToPILImage(),
                                transforms.Grayscale(num_output_channels=1),
                                transforms.ToTensor(),
                            ])

print("Load traindata..")
# for training

BATCH_SIZE = 400

traindata       = np.load('train.npy')
traindataset    = numpyDataset(traindata, transform)
trainloader     = DataLoader(traindataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=6)

print("Train data is ready")


Load traindata..
Train data is ready


In [10]:
size_img = 120 * 80
size_layer_1 = 100
size_layer_2 = 50
size_layer_3 = 12
size_layer_4 = 3

class autoencoder(nn.Module):

    def __init__(self) :
        super(autoencoder, self).__init__()
        
        self.encoder = nn.Sequential (
            nn.Conv2d(1, 6, kernel_size = 3),
            nn.ReLU(True),
            nn.Conv2d(6, 16,kernel_size= 5),
            nn.ReLU(True))
        self.decoder = nn.Sequential(
            nn.ConvTranspose2d(16,6,kernel_size= 5),
            nn.ReLU(True),
            nn.ConvTranspose2d(6,1,kernel_size= 3),
            nn.ReLU(True))

    def forward(self, x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x
    

    
max_epochs = 10000 # For prevent system down

#5 # 6
learning_rate = 1e-3

In [ ]:
model = autoencoder().cuda()
optimizer = torch.optim.SGD(
    model.parameters(), lr=learning_rate, weight_decay=1e-3,momentum=0.9)
criterion = nn.MSELoss()

epoch_history = []
epoch_loss_history = []
epoch_loss_std_history = []
exit_flag = 0

for epoch in range(max_epochs):
    # =========== Saving Origin imgs =========== #
    if epoch == 0 :
        for data in enumerate(traindata) :
            img = torch.tensor(data[1]).cuda()
            to_img = transforms.ToPILImage()
            pic_origin = to_img(img.view(120,80).cpu())
            pic_origin.save('./mlp_img/{}_origin.jpg'.format(data[0]))

    # ================= Train ===================
    loss_logs = []
    for data in enumerate(trainloader):
        
        order = data[0]
        img = data[1].clone().detach()
   
        img = img.cuda()
     
        # Create Noisy img
        
        # Random sigma between 0.01 - 0.04
        noise_level = torch.tensor(0.01) + torch.rand(1) * 0.03
        normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
        noise = normal_distrib.sample(img.shape).view(img.shape).cuda()
        
        noisy_img = img + noise
        
        # ===================forward=====================
        output = model(noisy_img)
        loss = criterion(output, img)
        # ===================backward====================
        optimizer.zero_grad()
        loss.backward()
        loss_logs.append(loss.item())
        optimizer.step()
        
        
        # ================= This code for Debug ====================#
        if epoch == 1 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[2].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(21))
                
                pic_origin = to_img(img[6].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(25))
            else :
                to_img = transforms.ToPILImage()
                pic_origin = to_img(img[3].view(120,80).cpu())
                pic_origin.save('./train_img/{}_origin.jpg'.format(42))
            
        '''        
        if order == 2 or order == 4 :
            if order == 2 :
                to_img = transforms.ToPILImage()
                
                ## 21
                pic_input = to_img(noisy_img[2].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(21,epoch))
                 
                pic_output = to_img(output[2].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(21,epoch))

                ## 25
                pic_input = to_img(noisy_img[6].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(25,epoch))
                
                
                pic_output = to_img(output[6].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(25,epoch))
            else :
                ## 42
                to_img = transforms.ToPILImage()
                pic_input = to_img(noisy_img[3].view(120,80).cpu())
                pic_input.save('./train_img/{}_{}_input.jpg'.format(42,epoch))
                
                
                pic_output = to_img(output[3].view(120,80).cpu())
                pic_output.save('./train_img/{}_{}_output.jpg'.format(42,epoch))
        '''
    epoch_loss = np.mean(loss_logs)
    epoch_loss_std = np.std(loss_logs)
        
    # ===================log========================
    print('[epoch {}], loss: (training) {:.10f}'
          .format(epoch, epoch_loss))
    
    
    # ================ loss diff check ============
    epoch_loss_history.append(epoch_loss)
    epoch_loss_std_history.append(epoch_loss_std)
    epoch_history.append(epoch)
    
    if epoch != 0 :
        loss_diff = abs(epoch_loss_history[epoch] - epoch_loss_history[epoch-1])
        if loss_diff <= 0.0000001 and epoch_loss_history[epoch-1] < 0.0004 :
            exit_flag += 1
            if exit_flag >= 2 :
                break
        else :
            exit_flag = 0
        

[epoch 0], loss: (training) 0.1470126733
[epoch 1], loss: (training) 0.0719894492
[epoch 2], loss: (training) 0.0214485362
[epoch 3], loss: (training) 0.0217661643
[epoch 4], loss: (training) 0.0173324089
[epoch 5], loss: (training) 0.0171624693
[epoch 6], loss: (training) 0.0164761721
[epoch 7], loss: (training) 0.0162525871
[epoch 8], loss: (training) 0.0159885535
[epoch 9], loss: (training) 0.0157761224
[epoch 10], loss: (training) 0.0155446167
[epoch 11], loss: (training) 0.0153444006
[epoch 12], loss: (training) 0.0151445778
[epoch 13], loss: (training) 0.0149209962
[epoch 14], loss: (training) 0.0147308032
[epoch 15], loss: (training) 0.0145320196
[epoch 16], loss: (training) 0.0143496111
[epoch 17], loss: (training) 0.0141560512
[epoch 18], loss: (training) 0.0139882880
[epoch 19], loss: (training) 0.0138131938
[epoch 20], loss: (training) 0.0136364861
[epoch 21], loss: (training) 0.0134649987
[epoch 22], loss: (training) 0.0133057022
[epoch 23], loss: (training) 0.0131366065
[e

[epoch 194], loss: (training) 0.0035205568
[epoch 195], loss: (training) 0.0035211988
[epoch 196], loss: (training) 0.0035030845
[epoch 197], loss: (training) 0.0034998464
[epoch 198], loss: (training) 0.0034996821
[epoch 199], loss: (training) 0.0034887214
[epoch 200], loss: (training) 0.0034782593
[epoch 201], loss: (training) 0.0034784826
[epoch 202], loss: (training) 0.0034606736
[epoch 203], loss: (training) 0.0034668233
[epoch 204], loss: (training) 0.0034658485
[epoch 205], loss: (training) 0.0034569583
[epoch 206], loss: (training) 0.0034524902
[epoch 207], loss: (training) 0.0034350361
[epoch 208], loss: (training) 0.0034304435
[epoch 209], loss: (training) 0.0034237388
[epoch 210], loss: (training) 0.0034189927
[epoch 211], loss: (training) 0.0034119844
[epoch 212], loss: (training) 0.0034110175
[epoch 213], loss: (training) 0.0034013528
[epoch 214], loss: (training) 0.0033964758
[epoch 215], loss: (training) 0.0033976694
[epoch 216], loss: (training) 0.0033870919
[epoch 217]

[epoch 385], loss: (training) 0.0026040074
[epoch 386], loss: (training) 0.0026063068
[epoch 387], loss: (training) 0.0025988162
[epoch 388], loss: (training) 0.0025935050
[epoch 389], loss: (training) 0.0025741643
[epoch 390], loss: (training) 0.0025878692
[epoch 391], loss: (training) 0.0025869399
[epoch 392], loss: (training) 0.0025731666
[epoch 393], loss: (training) 0.0025678923
[epoch 394], loss: (training) 0.0025543812
[epoch 395], loss: (training) 0.0025557486
[epoch 396], loss: (training) 0.0025511568
[epoch 397], loss: (training) 0.0025600731
[epoch 398], loss: (training) 0.0025528101
[epoch 399], loss: (training) 0.0025667536
[epoch 400], loss: (training) 0.0025587787
[epoch 401], loss: (training) 0.0025519893
[epoch 402], loss: (training) 0.0025487116
[epoch 403], loss: (training) 0.0025462508
[epoch 404], loss: (training) 0.0025383318
[epoch 405], loss: (training) 0.0025530591
[epoch 406], loss: (training) 0.0025390598
[epoch 407], loss: (training) 0.0025269941
[epoch 408]

In [ ]:
print(int(torch.randint(4,(1,1))))

In [ ]:
test = []
for data in enumerate(trainloader):
        
    order = data[0]
    img = data[1].clone().detach()

    img = img.cuda()

    # Create Noisy img

    # Random sigma between 0.01 - 0.04
    
    noise_level = torch.tensor(0.01) + torch.rand(1) * 0.03
    normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
    noise = normal_distrib.sample(img.shape).view(img.shape).cuda()

    noisy_img = img + noise

    # ===================forward=====================
    output = model(noisy_img)
    loss = criterion(output, img)
    # ===================backward====================
    optimizer.zero_grad()
    test.append(loss.item())
test = np.array(test)
test_mean = test.mean()
print("Test loss : {:10f}".format(test_mean))

test = []

for data in enumerate(trainloader):
        
    order = data[0]
    img = data[1].clone().detach()

    img = img.cuda()

    # Create Noisy img

    # Random sigma between 0.01 - 0.04
    
    noise_level = torch.tensor(0.01) + int(torch.randint(4,(1,1)))* 0.01
    normal_distrib = torch.distributions.Normal(torch.tensor([0.0]),noise_level)
    noise = normal_distrib.sample(img.shape).view(img.shape).cuda()

    noisy_img = img + noise

    # ===================forward=====================
    output = model(noisy_img)
    loss = criterion(output, img)
    # ===================backward====================
    optimizer.zero_grad()
    test.append(loss.item())

test = np.array(test)
test_mean = test.mean()
print("Test loss : {:10f}".format(test_mean))


In [ ]:

graph_x = []
graph_y = []

graph = plt.plot(epoch_history,epoch_loss_history, color = 'orange', label = 'Train Loss')
plt.title("Loss")
plt.legend(['Train Loss'])
plt.show()


graph_x = []
graph_y = []

graph = plt.plot(epoch_history,epoch_loss_std_history, color = 'red', label = 'Train Loss')
plt.title("Std")
plt.legend(['Train Loss Std'])
plt.show()


In [ ]:


# for testing
testdata        = np.load('test.npy')
testdataset     = numpyDataset(testdata, transform)
testloader      = DataLoader(testdataset, batch_size=1, shuffle=False, num_workers=2)

result_for_submit = None    # this is for submit file

for batch_idx, data in enumerate(testloader):

    result_of_test = model(data.cuda())
    
    #print(result_of_test)
    
    if batch_idx == 0:
        result_for_submit = result_of_test
    else:
        try:
            result_for_submit = torch.cat([result_for_submit, result_of_test], dim=0)

        except RuntimeError:
            transposed = torch.transpose(result_of_test, 2, 3)
            result_for_submit = torch.cat([result_for_submit, transposed], dim=0)

            
       
result_for_submit = result_for_submit.cpu()
     


# the submit_file.shape must be (400,1,120,80) 
submit_file = result_for_submit.detach().numpy()
print("submit_file.shape :",submit_file.shape)

np.save('jesoonkang-2.npy', submit_file)
print("Test Result Save Success")

